# 🤖 Machine Learning FX Carry Trading Strategy
## Training, Validation, and Deployment

**Date**: November 6, 2025  
**Objective**: Train ensemble ML models (RF, XGB, LSTM) to predict FX returns

---

## 📦 Setup & Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Import our ML modules
import sys
sys.path.append('./ml_fx')

from ml_fx.data_loader import MLDataLoader
from ml_fx.feature_engineer import FeatureEngineer
from ml_fx.ml_models import MLEnsemble
from ml_fx.ml_strategy import MLFXStrategy

# Load environment
from dotenv import load_dotenv
import os
load_dotenv()

print("✅ Setup complete!")

✅ Setup complete!


## 📊 Step 1: Load & Explore Data

In [3]:
# Initialize data loader
fred_key = os.getenv('FRED_API_KEY', 'b4a18aac3a462b6951ee89d9fef027cb')

loader = MLDataLoader(fred_api_key=fred_key)

# Load comprehensive dataset (using 2018+ for faster training)
data = loader.load_all_data(start_date='2018-01-01')

print(f"\n📊 Dataset Summary:")
print(f"   Shape: {data.shape}")
print(f"   Date range: {data.index[0]} to {data.index[-1]}")
print(f"   Columns: {len(data.columns)}")
print(f"\n   Sample:")
print(data.tail())

📊 Loading comprehensive dataset for ML...
  ├─ Loading FX rates...
  ├─ Loading interest rates...
  ├─ Loading market data (equities, VIX, etc.)...
  ├─ Loading macro data (GDP, inflation, etc.)...
  ├─ Loading commodity data...


IndexError: index -1 is out of bounds for axis 0 with size 0

## 🔧 Step 2: Feature Engineering

In [ ]:
# Initialize feature engineer
currencies = ['AUD', 'BRL', 'CAD', 'CHF', 'EUR', 'GBP', 'JPY', 'MXN']
fe = FeatureEngineer(currencies=currencies)

# Create features
features = fe.create_all_features(data)

print(f"\n🔧 Feature Engineering Results:")
print(f"   Total features: {len(features.columns)}")
print(f"   Sample size: {len(features)} rows")

# Show feature groups
feature_groups = fe.get_feature_groups(features)
print(f"\n   Feature breakdown:")
for name, cols in feature_groups.items():
    print(f"     {name:15s}: {len(cols):3d} features")

## 🤖 Step 3: Train ML Models for EUR (Demo)
### Let's train one currency first to see how it works

In [ ]:
# Initialize ensemble
ensemble = MLEnsemble(model_dir="./ml_models")

# Prepare EUR data
currency = 'EUR'
fx_col = f'{currency}_USD'

if fx_col in data.columns:
    # Create target: predict next 21 days return
    target = data[fx_col].pct_change(21).shift(-21)
    
    # Align features and target
    valid_idx = features.index.intersection(target.dropna().index)
    X = features.loc[valid_idx]
    y = target.loc[valid_idx]
    
    print(f"📊 Training data for {currency}:")
    print(f"   Features: {X.shape}")
    print(f"   Target: {y.shape}")
    
    # Train models
    results = ensemble.train(
        X, y, 
        currency=currency,
        validation_split=0.2,
        optimize=True
    )
    
    # Save models
    ensemble.save_models(currency)
    
    print(f"\n✅ {currency} training complete!")
else:
    print(f"❌ No data for {currency}")

## 🎯 Step 4: View EUR Feature Importance

In [ ]:
# Get top features for EUR
top_features = ensemble.get_feature_importance('EUR', top_n=20)

# Plot
plt.figure(figsize=(12, 8))
plt.barh(range(len(top_features)), top_features['avg_importance'][::-1])
plt.yticks(range(len(top_features)), top_features['feature'].iloc[::-1])
plt.xlabel('Average Importance (RF + XGB)')
plt.title('EUR - Top 20 Most Important Features')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n📊 Top 20 Features for EUR:")
print(top_features.to_string(index=False))

## 🚀 Step 5: Train ALL Currencies
### This will take 20-30 minutes - be patient!

In [ ]:
# Initialize ML Strategy
strategy = MLFXStrategy(
    fred_api_key=fred_key,
    currencies=currencies
)

# Train all currencies
print("🚀 Starting full training... (20-30 minutes)\n")
print("☕ Grab a coffee - this will take a while!\n")

all_results = strategy.train_all_currencies(
    start_date='2018-01-01',
    validation_split=0.2,
    walk_forward=False
)

## 📊 Step 6: Performance Summary

In [ ]:
# Extract performance metrics
performance_data = []

for currency in currencies:
    if currency in all_results:
        res = all_results[currency]
        performance_data.append({
            'Currency': currency,
            'RF_R2': res['rf']['r2'],
            'XGB_R2': res['xgb']['r2'],
            'LSTM_R2': res['lstm']['r2'],
            'Ensemble_R2': res['ensemble']['r2']
        })

perf_df = pd.DataFrame(performance_data)

# Visualize
fig, ax = plt.subplots(figsize=(14, 6))

x = np.arange(len(perf_df))
width = 0.2

ax.bar(x - 1.5*width, perf_df['RF_R2'], width, label='Random Forest', alpha=0.8)
ax.bar(x - 0.5*width, perf_df['XGB_R2'], width, label='XGBoost', alpha=0.8)
ax.bar(x + 0.5*width, perf_df['LSTM_R2'], width, label='LSTM', alpha=0.8)
ax.bar(x + 1.5*width, perf_df['Ensemble_R2'], width, label='Ensemble', alpha=0.8, color='gold')

ax.set_xlabel('Currency')
ax.set_ylabel('R² Score')
ax.set_title('Model Performance Comparison (R² Score)')
ax.set_xticks(x)
ax.set_xticklabels(perf_df['Currency'])
ax.legend()
ax.grid(axis='y', alpha=0.3)
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax.axhline(y=0.05, color='green', linestyle='--', linewidth=1, alpha=0.5, label='Good threshold')

plt.tight_layout()
plt.show()

print("\n📊 Performance Summary:")
print(perf_df.to_string(index=False))
print(f"\n📈 Average Ensemble R²: {perf_df['Ensemble_R2'].mean():.4f}")

## 🔮 Step 7: Generate Live Trading Signals

In [ ]:
# Generate current signals
signals = strategy.generate_signals()

# Convert to positions
positions = strategy.generate_positions(
    signals=signals,
    capital=100000,
    max_position_size=0.25,
    risk_scale=1.0
)

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

# Signals
signal_df = pd.DataFrame(list(signals.items()), columns=['Currency', 'Signal'])
signal_df = signal_df.sort_values('Signal')

colors = ['red' if x < 0 else 'green' for x in signal_df['Signal']]
ax1.barh(signal_df['Currency'], signal_df['Signal'], color=colors, alpha=0.7)
ax1.set_xlabel('Signal Strength (-1 to +1)')
ax1.set_title('ML Trading Signals (Current)')
ax1.axvline(x=0, color='black', linestyle='-', linewidth=1)
ax1.grid(axis='x', alpha=0.3)

# Positions
position_df = pd.DataFrame(list(positions.items()), columns=['Currency', 'Position'])
position_df = position_df.sort_values('Position')

colors = ['red' if x < 0 else 'green' for x in position_df['Position']]
ax2.barh(position_df['Currency'], position_df['Position'], color=colors, alpha=0.7)
ax2.set_xlabel('Position Size (USD)')
ax2.set_title('Recommended Positions ($100k Capital)')
ax2.axvline(x=0, color='black', linestyle='-', linewidth=1)
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🎯 Trading Signals:")
print(signal_df.to_string(index=False))
print("\n💰 Recommended Positions:")
print(position_df.to_string(index=False))

## ✅ Step 8: Summary & Next Steps

In [ ]:
print("\n" + "="*70)
print("✅ ML TRAINING COMPLETE")
print("="*70)

print(f"\n📊 Training Summary:")
print(f"   Currencies trained: {len([c for c in currencies if c in all_results])}")
print(f"   Features engineered: {len(features.columns)}")
print(f"   Training period: {data.index[0].date()} to {data.index[-1].date()}")

print(f"\n🎯 Performance:")
avg_r2 = perf_df['Ensemble_R2'].mean()
print(f"   Average Ensemble R²: {avg_r2:.4f}")

if avg_r2 > 0.05:
    print("   🎉 EXCELLENT! Models show strong predictive power")
elif avg_r2 > 0:
    print("   ✅ GOOD! Models beat random baseline")
else:
    print("   ⚠️  Models need improvement - try more data or features")

print(f"\n🚀 Next Steps:")
print("   1. ✅ Models trained and saved")
print("   2. ✅ Live signals generated")
print("   3. 🔄 Integrate with live_trading_system.py")
print("   4. 📄 Start paper trading")
print("   5. 📊 Monitor performance vs baseline")

print(f"\n📁 Models saved in: ./ml_models/")
print(f"   Ready to load with: strategy.load_trained_models()")

print("\n🎊 Congratulations! You have a working ML trading system!")